<a href="https://colab.research.google.com/github/oallanfarias/pipelines/blob/main/pipelines.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
#ATUALIZAR O PIP
!pip install --upgrade pip

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 2.0 MB 9.7 MB/s 
  Attempting uninstall: pip
    Found existing installation: pip 21.1.3
    Uninstalling pip-21.1.3:
      Successfully uninstalled pip-21.1.3


In [ ]:
#INSTALAR O APACHE_BEAM[INTERACTIVE]
!pip install apache_beam[interactive]

In [ ]:
#INSTALAR O APACHE_BEAM[GCP]
!pip install apache_beam[gcp]

In [1]:
#IMPORTAR A BIBLIOTECA DO APACHE BEAM
import apache_beam as beam

#Ler e exibir leitura

In [ ]:
#TODA | É UM PASSO DA PIPELINE
#O APACHE BEAM NÃO FAZ DIFERENÇAS ENTRE OS FORMATOS DE ARQUIVOS. PARA ELE, TODOS SÃO ARQUIVOS DE TEXTO
import apache_beam as beam

p1 = beam.Pipeline()

voos = (
    p1
    |'Extrair do CSV'>> beam.io.ReadFromText('/content/drive/MyDrive/Datasets/voos.csv', skip_header_lines = 0) #ABRIU O ARQUIVO PULA A PRIMEIRA (0) LINHA 
    |'Separador do csv'>> beam.Map(lambda record: record.split(',')) #TODA VEZ QUE VEM A VIRGULA ELA SEPARA A INFORMAÇÃO
    |'Exibir o resultado'>> beam.Map(print) #PRINTOU O RESULTADO
)

p1.run()

In [5]:
import apache_beam as beam

p1 = beam.Pipeline()

voos = (
    p1
    |'Extrair do CSV'>> beam.io.ReadFromText('/content/drive/MyDrive/Datasets/voos.csv', skip_header_lines = 0) #ABRIU O ARQUIVO PULA A PRIMEIRA (0) LINHA 
    |'Separador do csv'>> beam.Map(lambda record: record.split(',')) #TODA VEZ QUE VEM A VIRGULA ELA SEPARA A INFORMAÇÃO
    #|'Exibir o resultado'>> beam.Map(print) #PRINTOU A SAIDA
    |'Gravar o resultado'>> beam.io.WriteToText('/content/drive/MyDrive/Datasets/saida.txt') #OU SALVA O ARQUIVO COM O COMANDO 'WriteToText' OU PRINTA COM O 'Map(print)'
)

p1.run()

#Filtros

##Filtros sobre String

In [ ]:
#FILTRANDO POR AEROPORTO DE ORIGEM SFO
import apache_beam as beam

p1 = beam.Pipeline()

voos = (
    p1
    |'Extrair do CSV'>> beam.io.ReadFromText('/content/drive/MyDrive/Datasets/voos.csv', skip_header_lines = 0) #abriu o arquivo, pula a primeira linha 
    |'Separador do csv'>> beam.Map(lambda record: record.split(',')) #toda vez que vem a virgula ele separou
    |'Filtrar aero'>> beam.Filter(lambda record: record[4] == 'SFO') # O número em record[] define a coluna.
    |'Exibir o resultado'>> beam.Map(print) #printou
    #|'Gravar o resultado'>> beam.io.WriteToText('/content/drive/MyDrive/Datasets/saida.txt') #salvar no arquivo, ou printa ou guarda
)

p1.run()

In [ ]:
# Filtrando por saída atrasada
import apache_beam as beam

p1 = beam.Pipeline()

voos = (
    p1
    |'Extrair do CSV'>> beam.io.ReadFromText('/content/drive/MyDrive/Datasets/voos.csv',skip_header_lines = 1 )
    |'Separador do csv' >> beam.Map(lambda record: record.split(','))
    |'Filtrar aero'>> beam.Filter(lambda record: float(record[8])>0)
    |'Exibir o resultado'>> beam.Map(print)
    #|'Gravar o resultado'>> beam.io.WriteToText('saida2.txt')
)

p1.run()

#Agrupamento

In [ ]:
# Filtrando por saída atrasada com total de minutos por aeroporto de origem
import apache_beam as beam

p1 = beam.Pipeline()

qnt_tempo = (
    p1
    |'Extrair do CSV'>> beam.io.ReadFromText('/content/drive/MyDrive/Datasets/voos.csv',skip_header_lines = 1 )
    |'Separador do csv' >> beam.Map(lambda record: record.split(','))
    |'Filtrar aero'>> beam.Filter(lambda record: float(record[8])>0)
    |'Agregar as colunas'>> beam.Map(lambda record: (record[4],float(record[8])))
    |'Construir uma nova tabela' >> beam.CombinePerKey(sum)
    |'Exibir o resultado'>> beam.Map(print)
    #|'Gravar o resultado'>> beam.io.WriteToText('saida2.txt')
)

p1.run()

In [ ]:
# Filtrando por saída atrasada com total de vezes por aeroporto de origem
import apache_beam as beam

p1 = beam.Pipeline()

voos = (
    p1
    |'Extrair do CSV'>> beam.io.ReadFromText('/content/drive/MyDrive/Datasets/voos.csv',skip_header_lines = 1 )
    |'Separador do csv' >> beam.Map(lambda record: record.split(','))
    |'Filtrar aero'>> beam.Filter(lambda record: float(record[8])>0)
    |'Agregar as colunas'>> beam.Map(lambda record: (record[4],float(record[8])))
    |'Construir uma nova tabela por qtd' >> beam.combiners.Count.PerKey(sum)
    |'Exibir o resultado'>> beam.Map(print)
    #|'Gravar o resultado'>> beam.io.WriteToText('saida2.txt')
)

p1.run()

#Co Group By Key

In [12]:
'''
Podemos utilizar diferentes pipelines para combinar e organizar informações.
Importante: a definição de cada pipeline nunca pode ser igual. Por isso, neste exemplo, inserimos o número 1.
A tabela reune as nformações separadas por chaves.
'''
#Criar nova tabela com dados agregados de duas pipes
import apache_beam as beam

p1 = beam.Pipeline()

qte_tempo = (
    p1
    |'Extrair do CSV'>> beam.io.ReadFromText('/content/drive/MyDrive/Datasets/voos.csv',skip_header_lines = 1 )
    |'Separador do csv' >> beam.Map(lambda record: record.split(','))
    |'Filtrar aero'>> beam.Filter(lambda record: float(record[8])>0)
    |'Agregar as colunas'>> beam.Map(lambda record: (record[4],float(record[8])))
    |'Construir uma nova tabela por minuto' >> beam.CombinePerKey(sum)
    #|'Exibir o resultado'>> beam.Map(print)
    #|'Gravar o resultado'>> beam.io.WriteToText('voos_minutos.txt')
)

qte_vezes = (
    p1
    |'1Extrair do CSV'>> beam.io.ReadFromText('/content/drive/MyDrive/Datasets/voos.csv',skip_header_lines = 1 )
    |'1Separador do csv' >> beam.Map(lambda record: record.split(','))
    |'1Filtrar aero'>> beam.Filter(lambda record: float(record[8])>0)
    |'1Agregar as colunas'>> beam.Map(lambda record: (record[4],float(record[8])))
    |'1Construir uma nova tabela por qtd' >> beam.combiners.Count.PerKey()
    #|'Exibir o resultado'>> beam.Map(print)
    #|'Gravar o resultado'>> beam.io.WriteToText('voos_minutos.txt')
)

tabela = (
    {'Quantidade_minutos':qte_tempo,'Numero_de_atrasos':qte_vezes}
    |'Agrupar as pernas' >> beam.CoGroupByKey()
    #|'Resultado' >>beam.Map(print)
    |'Gravar o resultado'>> beam.io.WriteToText('/content/drive/MyDrive/Datasets/novatabela.txt')
)
p1.run()